# ML Project Assignment

### Predicting players rating 

In this project you are going to predict the overall rating of soccer player based on their attributes such as 'crossing', 'finishing etc.

The dataset you are going to use is from European Soccer Database  (https://www.kaggle.com/hugomathien/soccer) has more than 25,000 matches and more  
than 10,000 players for European professional soccer seasons from 2008 to 2016.

Download the data in the same folder and run the following commmand to get it in the environment 

About the Dataset 

The ultimate Soccer database for data analysis and machine learning 

The dataset comes in the form of an SQL database and contains statistics of about 25,000 football  matches, from the top football league of 11 European Countries. It covers seasons from 2008 to  2016 and contains match statistics (i.e: scores, corners, fouls etc...) as well as the team  
formations, with player names and a pair of coordinates to indicate their position on the pitch. 

* +25,000 matches 
* +10,000 players 
* 11 European Countries with their lead championship 
* Seasons 2008 to 2016 
* Players and Teams' attributes* sourced from EA Sports' FIFA video game series, including  the weekly updates 
* Team line up with squad formation (X, Y coordinates) 
* Betting odds from up to 10 providers 
* Detailed match events (goal types, possession, corner, cross, fouls, cards etc...) for +10,000 matches 

The dataset also has a set of about 35 statistics for each player, derived from EA Sports' FIFA  video games. It is not just the stats that come with a new version of the game but also the weekly  updates. So for instance if a player has performed poorly over a period of time and his stats get  impacted in FIFA, you would normally see the same in the dataset. 

### Python skills required to complete this project 
#### SQL: 
The data is in SQL database so students need to retrieve using a query language. They also need to know how to connect the SQL database with python. The library we are using for this in 'sqlite3'. 

SQLite3 can be integrated with Python using sqlite3 module, which was written by Gerhard Haring.  It provides an SQL interface compliant with the DB-API 2.0 specification described by PEP 249.  You do not need to install this module separately because it is shipped by default along with Python  version 2.5.x onwards. 

To use sqlite3 module, you must first create a connection object that represents the database  and then optionally you can create a cursor object, which will help you in executing all the SQL  statements.

#### Pandas: 
Pandas is an open-source, BSD-licensed Python library providing high-performance, easy-to-use  data structures and data analysis tools for the Python programming language. Python with  Pandas is used in a wide range of fields including academic and commercial domains including  finance, economics, Statistics, analytics, etc.In this tutorial, we will learn the various features of  Python Pandas and how to use them in practice.

#### Scikit Learn 
Scikit-learn provides a range of supervised and unsupervised learning algorithms via a  
consistent interface in Python.

The library is built upon the SciPy (Scientific Python) that must be installed before you can  use scikit-learn. This stack that includes: 
* NumPy: Base n-dimensional array package 
* SciPy: Fundamental library for scientific computing 
* Matplotlib: Comprehensive 2D/3D plotting 
* IPython: Enhanced interactive console 
* Sympy: Symbolic mathematics 
* Pandas: Data structures and analysis 

Extensions or modules for SciPy care conventionally named SciKits. As such, the module  
provides learning algorithms and is named scikit-learn.

The vision for the library is a level of robustness and support required for use in production systems.  This means a deep focus on concerns such as easy of use, code quality, collaboration,  
documentation and performance.

### Machine Learning skills required to complete the project 
#### Supervised learning 
Supervised learning deals with learning a function from available training data. A supervised  learning algorithm analyzes the training data and produces an inferred function, which can be  used for mapping new examples. 

#### Regression 
Regression is a parametric technique used to predict continuous (dependent) variable given a set  of independent variables. It is parametric in nature because it makes certain assumptions  
(discussed next) based on the data set. If the data set follows those assumptions, regression gives  incredible results. 

#### Model evaluation 
Student must know how to judge a model on unseen data. What metric to select to judge  
the performance 

Let's get started..... 

### Import Libraries

In [1]:
!pip install xgboost

import sqlite3
import numpy as np
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from xgboost import plot_importance

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer as Imputer
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit, RandomizedSearchCV
from sklearn.pipeline import make_pipeline

import pickle

In [2]:
cnx = sqlite3.connect('database.sqlite')

In [3]:
dd = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", cnx)

In [4]:
print(dd)

                name
0    sqlite_sequence
1  Player_Attributes
2             Player
3              Match
4             League
5            Country
6               Team
7    Team_Attributes


In [5]:
df = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)

In [6]:
df.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [7]:
target = df.pop('overall_rating')

In [8]:
type(target)

pandas.core.series.Series

In [9]:
df.shape

(183978, 41)

In [10]:
target.head()

0    67.0
1    67.0
2    62.0
3    61.0
4    61.0
Name: overall_rating, dtype: float64

## Imputing target funtion :

In [11]:
target.isnull().values.sum()

836

there are 836 missing value present in target function.

In [12]:
target.describe()

count    183142.000000
mean         68.600015
std           7.041139
min          33.000000
25%          64.000000
50%          69.000000
75%          73.000000
max          94.000000
Name: overall_rating, dtype: float64

In [13]:
y = target.fillna(target.mean())

In [14]:
y.isnull().values.any()

False

## Data Exploration :

In [15]:
df.columns

Index(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'potential',
       'preferred_foot', 'attacking_work_rate', 'defensive_work_rate',
       'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys',
       'dribbling', 'curve', 'free_kick_accuracy', 'long_passing',
       'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions',
       'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'aggression', 'interceptions', 'positioning', 'vision', 'penalties',
       'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving',
       'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes'],
      dtype='object')

In [16]:
for col in df.columns:
    unique_cat = len(df[col].unique())
    print("{col}--> {unique_cat}..{typ}".format(col=col, unique_cat=unique_cat, typ=df[col].dtype))

id--> 183978..int64
player_fifa_api_id--> 11062..int64
player_api_id--> 11060..int64
date--> 197..object
potential--> 57..float64
preferred_foot--> 3..object
attacking_work_rate--> 9..object
defensive_work_rate--> 20..object
crossing--> 96..float64
finishing--> 98..float64
heading_accuracy--> 97..float64
short_passing--> 96..float64
volleys--> 94..float64
dribbling--> 98..float64
curve--> 93..float64
free_kick_accuracy--> 98..float64
long_passing--> 96..float64
ball_control--> 94..float64
acceleration--> 87..float64
sprint_speed--> 86..float64
agility--> 82..float64
reactions--> 79..float64
balance--> 82..float64
shot_power--> 97..float64
jumping--> 80..float64
stamina--> 85..float64
strength--> 83..float64
long_shots--> 97..float64
aggression--> 92..float64
interceptions--> 97..float64
positioning--> 96..float64
vision--> 98..float64
penalties--> 95..float64
marking--> 96..float64
standing_tackle--> 96..float64
sliding_tackle--> 95..float64
gk_diving--> 94..float64
gk_handling--> 91..

we can see only four features have the type 'object'. here the feature named 'date' has no significance in this problem so can ignore it and perform one hot encoding on the rest of 3 features.

In [17]:
dummy_df = pd.get_dummies(df, columns=['preferred_foot', 'attacking_work_rate', 'defensive_work_rate'])
dummy_df.head()

,id,player_fifa_api_id,player_api_id,date,potential,crossing,finishing,heading_accuracy,short_passing,volleys,...,defensive_work_rate_9,defensive_work_rate__0,defensive_work_rate_ean,defensive_work_rate_es,defensive_work_rate_high,defensive_work_rate_low,defensive_work_rate_medium,defensive_work_rate_o,defensive_work_rate_ormal,defensive_work_rate_tocky
0,1,218353,505942,2016-02-18 00:00:00,71.0,49.0,44.0,71.0,61.0,44.0,...,0,0,0,0,0,0,1,0,0,0
1,2,218353,505942,2015-11-19 00:00:00,71.0,49.0,44.0,71.0,61.0,44.0,...,0,0,0,0,0,0,1,0,0,0
2,3,218353,505942,2015-09-21 00:00:00,66.0,49.0,44.0,71.0,61.0,44.0,...,0,0,0,0,0,0,1,0,0,0
3,4,218353,505942,2015-03-20 00:00:00,65.0,48.0,43.0,70.0,60.0,43.0,...,0,0,0,0,0,0,1,0,0,0
4,5,218353,505942,2007-02-22 00:00:00,65.0,48.0,43.0,70.0,60.0,43.0,...,0,0,0,0,0,0,1,0,0,0


In [18]:
X = dummy_df.drop(['id', 'date'], axis=1)

***
## Feature selection :

* As tree model doesn't gets affected by missing values present in data set. but feature selection by `SelectFromModel` can not be done on datasets that carries null value. Therefore, we should also perform imputation on dataset. 

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [20]:
#imputing null value of each column with the mean of that column
imput = Imputer()
X_train = imput.fit_transform(X_train)
X_test = imput.fit_transform(X_test)

In [21]:
#finding feature_importance for feature selection. from it we'll be able to decide threshold value
model = XGBRegressor()
model.fit(X_train, y_train)
print(model.feature_importances_)

[0.0112063  0.01328359 0.16840546 0.01479726 0.00693286 0.02923344
 0.02684482 0.00141261 0.01633997 0.00142012 0.00089428 0.00336454
 0.05596948 0.00311885 0.00745351 0.00146221 0.3133247  0.00111532
 0.02715905 0.00230193 0.00448559 0.02142001 0.00503636 0.0047115
 0.01690763 0.01669931 0.00246637 0.00140508 0.02284729 0.05204851
 0.00592539 0.02880221 0.02403816 0.00703669 0.03812633 0.022119
 0.00037242 0.         0.00078061 0.00141978 0.00046924 0.00049927
 0.0007586  0.00086996 0.00040868 0.00098631 0.         0.
 0.         0.00204254 0.00125616 0.         0.00173628 0.
 0.0006331  0.         0.00087684 0.         0.         0.00092218
 0.00059731 0.00364412 0.00161083 0.         0.        ]


In [22]:
selection = SelectFromModel(model, threshold=0.01, prefit=True)

select_X_train = selection.transform(X_train)
select_X_test = selection.transform(X_test)

In [23]:
select_X_train.shape

(137983, 19)

## Training different models :

### 1. Linear Regression :

In [24]:
pipe = make_pipeline(StandardScaler(),             #preprocessing(standard scalling)
                     LinearRegression())           #estimator(linear regression)

#cv = ShuffleSplit(random_state=0)   #defining type of cross_validation(shuffle spliting)

param_grid = {'linearregression__n_jobs': [-1]}     #parameters for model tunning

grid = GridSearchCV(pipe, param_grid=param_grid, cv=10)

In [25]:
grid.fit(select_X_train, y_train)          #training 

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid={'linearregression__n_jobs': [-1]})

In [26]:
grid.best_params_

{'linearregression__n_jobs': -1}

In [27]:
lin_reg = pickle.dumps(grid)

### 2. Decision Tree :

In [28]:
pipe = make_pipeline(StandardScaler(),                  #preprocessing
                     DecisionTreeRegressor(criterion='mse', random_state=0))          #estimator

#cv = ShuffleSplit(n_splits=10, random_state=42)        #cross validation

param_grid = {'decisiontreeregressor__max_depth': [3, 5, 9, 13]}

grid = GridSearchCV(pipe, param_grid=param_grid, cv=10)

In [29]:
grid.fit(select_X_train, y_train)          #training 

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('decisiontreeregressor',
                                        DecisionTreeRegressor(random_state=0))]),
             param_grid={'decisiontreeregressor__max_depth': [3, 5, 9, 13]})

In [30]:
grid.best_params_

{'decisiontreeregressor__max_depth': 13}

In [31]:
Dectree_reg = pickle.dumps(grid)

### 3. Ranom Forest :

In [32]:
pipe = make_pipeline(StandardScaler(),
                     RandomForestRegressor(n_estimators=200, random_state=0))

#cv = ShuffleSplit(test_size=0.2, random_state=0)

param_grid = {'randomforestregressor__max_features':['sqrt', 'log2'],
              'randomforestregressor__max_depth':[9, 10]}                 

grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)

In [33]:
grid.fit(select_X_train, y_train)          #training 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('randomforestregressor',
                                        RandomForestRegressor(n_estimators=200,
                                                              random_state=0))]),
             param_grid={'randomforestregressor__max_depth': [9, 10],
                         'randomforestregressor__max_features': ['sqrt',
                                                                 'log2']})

In [34]:
grid.best_params_

{'randomforestregressor__max_depth': 10,
 'randomforestregressor__max_features': 'sqrt'}

In [35]:
Randfor_reg = pickle.dumps(grid)

### 4. Xgboost regressor :

In [36]:
pipe = make_pipeline(StandardScaler(),
                     XGBRegressor(n_estimators=200, random_state=42))

#cv = ShuffleSplit(n_splits=10, random_state=0)

param_grid = {'xgbregressor__max_depth': [5, 7],
              'xgbregressor__learning_rate': [0.1, 0.3]}

grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs= -1)

In [37]:
grid.fit(select_X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('xgbregressor',
                                        XGBRegressor(base_score=None,
                                                     booster=None,
                                                     colsample_bylevel=None,
                                                     colsample_bynode=None,
                                                     colsample_bytree=None,
                                                     gamma=None, gpu_id=None,
                                                     importance_type='gain',
                                                     interaction_constraints=None,
                                                     learning_rate=None,
                                                     max_delta_step=None,
                                                     max_depth=None,
                            

In [38]:
grid.best_params_

{'xgbregressor__learning_rate': 0.3, 'xgbregressor__max_depth': 7}

In [39]:
xgbreg = pickle.dumps(grid)

## <u>Comparision between different models</u> :

In [40]:
lin_reg = pickle.loads(lin_reg)
Dectree_reg = pickle.loads(Dectree_reg)
Randfor_reg = pickle.loads(Randfor_reg)
xgbreg = pickle.loads(xgbreg)

In [41]:
print("""Linear Regressor accuracy is {lin}
DecisionTree Regressor accuracy is {Dec}
RandomForest regressor accuracy is {ran}
XGBoost regressor accuracy is {xgb}""".format(lin=lin_reg.score(select_X_test, y_test),
                                                       Dec=Dectree_reg.score(select_X_test, y_test),
                                                       ran=Randfor_reg.score(select_X_test, y_test),
                                                       xgb=xgbreg.score(select_X_test, y_test)))

Linear Regressor accuracy is 0.8487289602119861
DecisionTree Regressor accuracy is 0.928215413573541
RandomForest regressor accuracy is 0.938632183726589
XGBoost regressor accuracy is 0.97282125349505


By accuracy comparision performed above we can say hear that XGBoost regressor gives better result than any other model. and it can predict the target function with approx 98% accuracy.